In [43]:
import requests
import json 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import functools

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests

In [44]:
### Price Information EUR-USD
BASE_URL = "https://www.alphavantage.co/query"
from_currency="EUR"
to_currency="USD"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-23,1.10050
1,2022-03-22,1.10280
2,2022-03-21,1.10180
3,2022-03-18,1.10500
4,2022-03-17,1.10888
...,...,...
95,2021-11-10,1.14781
96,2021-11-09,1.15924
97,2021-11-08,1.15863
98,2021-11-05,1.15660


In [45]:
conflict=pd.read_csv("conflict.csv")
conflict.date=conflict.date.astype("datetime64")
conflict

,date,volume
0,2022-01-29,14325
1,2022-01-30,80902
2,2022-01-31,85266
3,2022-02-01,85196
4,2022-02-02,92291
5,2022-02-03,100882
6,2022-02-04,92285
7,2022-02-05,75837
8,2022-02-06,66046
9,2022-02-07,81750


In [46]:
correl_test= pd.merge(conflict, data, how ='inner', on =['date'])

In [47]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: 1.303647
p-value: 0.996629
Critical values:
	1%: -4.069
	5%: -3.127
	10%: -2.702


In [48]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -2.441374
p-value: 0.130398
Critical values:
	1%: -4.223
	5%: -3.189
	10%: -2.730


In [49]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.114286
volume,-0.114286,1.000000


In [50]:
maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [51]:
correl_test= pd.merge(conflict, data, how ='inner', on =['date'])
flag=-1

if abs(correl_test.corr().iloc[1,0]) > 0.5:
    flag = 1
    
if flag==1:

    i=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["price"]["volume"]
    j=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["volume"]["price"]
    

    if i>j:
        highest = i
        lowest = j
    else:
        highest=j
        lowest=i
    
    if abs(highest/lowest)>2 :
        print(1)
        
    else :
        print(-1)

else:
    print(flag)

1


In [52]:
grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])

,volume,price
volume,1.0000,0.34
price,0.0844,1.00


#### TRY_JPY

In [53]:
### Price Information EUR-USD
BASE_URL = "https://www.alphavantage.co/query"
from_currency="TRY"
to_currency="JPY"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-23,8.136
1,2022-03-22,8.143
2,2022-03-21,8.045
3,2022-03-18,8.008
4,2022-03-17,8.015
...,...,...
95,2021-11-10,11.567
96,2021-11-09,11.580
97,2021-11-08,11.658
98,2021-11-05,11.652


In [54]:
correl_test= pd.merge(conflict, data, how ='inner', on =['date'])

In [55]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -1.518272
p-value: 0.524454
Critical values:
	1%: -3.809
	5%: -3.022
	10%: -2.651


In [56]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -2.961341
p-value: 0.038666
Critical values:
	1%: -3.859
	5%: -3.042
	10%: -2.661


In [57]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.457143
volume,-0.457143,1.000000


In [58]:
maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [59]:
flag=-1

if abs(correl_test.corr().iloc[1,0]) > 0.5:
    flag = 1
    
if flag==1:

    i=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["price"]["volume"]
    j=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["volume"]["price"]
    

    if i>j:
        highest = i
        lowest = j
    else:
        highest=j
        lowest=i
    
    if abs(highest/lowest)>2 :
        print(1)
        
    else :
        print(-1)

else:
    print(flag)

1


### CHF-CNY

In [74]:
### Price Information EUR-USD
BASE_URL = "https://www.alphavantage.co/query"
from_currency="CHF"
to_currency="CNY"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-23,6.8380
1,2022-03-22,6.8245
2,2022-03-21,6.8162
3,2022-03-18,6.8261
4,2022-03-17,6.7702
...,...,...
95,2021-11-10,6.9557
96,2021-11-09,7.0136
97,2021-11-08,6.9938
98,2021-11-05,7.0135


In [75]:
correl_test= pd.merge(conflict, data, how ='inner', on =['date'])

In [76]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -2.624992
p-value: 0.087941
Critical values:
	1%: -4.138
	5%: -3.155
	10%: -2.714


In [77]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -0.722882
p-value: 0.840733
Critical values:
	1%: -4.223
	5%: -3.189
	10%: -2.730


In [78]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.263158
volume,-0.263158,1.000000


In [79]:
maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [80]:
flag=-1

if abs(correl_test.corr().iloc[1,0]) > 0.5:
    flag = 1
    
if flag==1:

    i=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["price"]["volume"]
    j=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["volume"]["price"]
    

    if i>j:
        highest = i
        lowest = j
    else:
        highest=j
        lowest=i
    
    if abs(highest/lowest)>2 :
        print(1)
        
    else :
        print(-1)

else:
    print(flag)

1


In [81]:
correl_test.corr()

,volume,price
volume,1.000000,-0.565544
price,-0.565544,1.000000


In [84]:
### Price Information EUR-USD
BASE_URL = "https://www.alphavantage.co/query"
from_currency="GBP"
to_currency="NGN"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-23,548.88
1,2022-03-22,551.30
2,2022-03-21,547.21
3,2022-03-18,548.14
4,2022-03-17,546.84
...,...,...
95,2021-11-10,551.30
96,2021-11-09,557.58
97,2021-11-08,557.46
98,2021-11-05,554.74


In [85]:
correl_test= pd.merge(conflict, data, how ='inner', on =['date'])

In [86]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -2.840944
p-value: 0.052659
Critical values:
	1%: -4.138
	5%: -3.155
	10%: -2.714


In [87]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -0.603513
p-value: 0.870186
Critical values:
	1%: -4.223
	5%: -3.189
	10%: -2.730


In [88]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.311278
volume,-0.311278,1.000000


In [89]:
maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [90]:
flag=-1

if abs(correl_test.corr().iloc[1,0]) > 0.5:
    flag = 1
    
if flag==1:

    i=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["price"]["volume"]
    j=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["volume"]["price"]
    

    if i>j:
        highest = i
        lowest = j
    else:
        highest=j
        lowest=i
    
    if abs(highest/lowest)>2 :
        print(1)
        
    else :
        print(-1)

else:
    print(flag)

-1


In [91]:
(correl_test.corr().iloc[1,0])

-0.6816685749460392

In [92]:
grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])

,volume,price
volume,1.0000,0.468
price,0.2687,1.000


### FED

In [131]:
### Price Information EUR-USD
BASE_URL = "https://www.alphavantage.co/query"
from_currency="USD"
to_currency="GBP"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-23,0.75723
1,2022-03-22,0.75391
2,2022-03-21,0.75951
3,2022-03-18,0.75901
4,2022-03-17,0.76041
...,...,...
95,2021-11-10,0.74587
96,2021-11-09,0.73749
97,2021-11-08,0.73724
98,2021-11-05,0.74063


In [132]:
fed=pd.read_excel("fed_unfiltered.xlsx")

In [133]:
fed

,date,volume
0,2022-02-21,19230.0
1,2022-02-22,94198.0
2,2022-02-23,92729.0
3,2022-02-24,97809.0
4,2022-02-25,95038.0
5,2022-02-26,79346.0
6,2022-02-27,75681.0
7,2022-02-28,99038.0
8,2022-03-01,91803.0
9,2022-03-02,152247.0


In [134]:
correl_test= pd.merge(fed, data, how ='inner', on =['date'])

In [135]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -2.709240
p-value: 0.072453
Critical values:
	1%: -3.964
	5%: -3.085
	10%: -2.682


In [136]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -4.369664
p-value: 0.000336
Critical values:
	1%: -3.788
	5%: -3.013
	10%: -2.646


In [137]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.305477
volume,-0.305477,1.000000


In [138]:
flag=-1

if abs(correl_test.corr().iloc[1,0]) > 0.5:
    flag = 1
    
if flag==1:

    i=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["price"]["volume"]
    j=grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])["volume"]["price"]
    

    if i>j:
        highest = i
        lowest = j
    else:
        highest=j
        lowest=i
    
    if abs(highest/lowest)>2 :
        print(1)
        
    else :
        print(-1)

else:
    print(flag)

-1


In [139]:
correl_test.corr().iloc[1,0]

0.4121634213042972